In [ ]:
!rm -f watson_streaming_pipelines.py*
!wget https://raw.githubusercontent.com/raanonr/DSX/master/watson_streaming_pipelines.py

In [ ]:
# @hidden_cell
# The Object Storage credentials should NOT come from the 'Service credentials' page.
# Get them from the option in DSX Notebook to 'insert to code' from the Data -> Connections.
# Then add an entry for the service's PUBLIC endpoint.

cos_credentials_stage1 = {
  'iam_url':'https://iam.stage1.ng.bluemix.net/oidc/token',
  'api_key':'<REPLACE>',
  'resource_instance_id':'crn:v1:staging:public:cloud-object-storage:   <REPLACE>',
  'url':'https://s3-api.us-geo.objectstorage.uat.service.networklayer.com',
    'endpoint':'https://s3.us-west.objectstorage.uat.softlayer.net'
}
cos_credentials_prod = {
  'iam_url':'https://iam.ng.bluemix.net/oidc/token',
  'api_key':'<REPLACE>',
  'resource_instance_id':'crn:v1:bluemix:public:cloud-object-storage:   <REPLACE>',
  'url':'https://s3-api.us-geo.objectstorage.service.networklayer.com',
      "endpoint":"https://s3-api.us-geo.objectstorage.softlayer.net",
}

# The Message Hub credentials should come from the 'Service credentials' page.

mh_credentials_stage1 = {
  "kafka_brokers_sasl": [
    "kafka02-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka04-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka01-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka03-stage1.messagehub.services.us-south.bluemix.net:9093",
    "kafka05-stage1.messagehub.services.us-south.bluemix.net:9093"
  ],
  "user": "<REPLACE>",
  "password": "<REPLACE>"
}
mh_credentials_prod = {
  "kafka_brokers_sasl": [
    "kafka01-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka05-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka03-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka02-prod01.messagehub.services.us-south.bluemix.net:9093",
    "kafka04-prod01.messagehub.services.us-south.bluemix.net:9093"
  ],
  "user": "<REPLACE>",
  "password": "<REPLACE>"
}

In [ ]:
# Choose the credentials to use.

#cos_credentials = cos_credentials_stage1
cos_credentials = cos_credentials_prod

mh_credentials = mh_credentials_stage1
#mh_credentials = mh_credentials_prod

In [ ]:
# Load the yelp data

import watson_streaming_pipelines as stp

bucket_name = 'pyml'
object_name = 'yelp_academic_dataset_review_first200k.json'

data = stp.get_from_cloud_object_storage( 
            api_key = cos_credentials['api_key'],
            full_object_path = bucket_name + '/' + object_name, 
            auth_endpoint = cos_credentials['iam_url'],
            service_endpoint = cos_credentials['endpoint']
) 

# Verify
print(len(data))

In [ ]:
# Convert and decode the binary data and load the yelp List with the JSON objects

from io import StringIO
import json

yelp = []
for line in StringIO(data.decode("utf-8")):
    yelp.append( json.loads(line))

# Verify
print(yelp[0]['text'])

In [ ]:
# Create the Message Hub producer
producer = stp.create_messagehub_producer( username = mh_credentials['user'], password = mh_credentials['password'], kafka_brokers_sasl = mh_credentials['kafka_brokers_sasl'])

# Enter your topic (<REPLACE>)
mh_topic = 'testTopic1'

In [ ]:
# Send selected yelp fields to the MH topic

import time
print(len(yelp))
#print(yelp[0]['stars'])
n=0
for msg in yelp: #[:1000]:
#    print(msg['stars'])
    producer.send( mh_topic, { 'stars': msg['stars'], 'text': msg['text']})
    n += 1
    if (n % 1000) == 0:
        time.sleep(1)
        print(n, "", end="")